In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
#from langchain.schema import Document

from langchain_core.documents import Document

#from langchain_community.vectorstores import chroma
from langchain_chroma import Chroma
#utility imports
import numpy as np
from typing import List




In [6]:
sample_docs=[
"""
1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.
Connectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).
Key Contacts: 7022588703 / 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, 
offering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.
 For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B.Tech students and 
 average packages around 15 LPA for top branches. The Corporate and Industry Relations (CIR) cell is particularly active here, 
 facilitating recruitment with global giants like Amazon, SAP Labs, and Walmart Global Tech. While smaller in size compared to Coimbatore, 
 the campus provides modern laboratories, design tools, and clean, hygiene-focused hostels that are highly rated by students


""",

"""
2. Coimbatore Campus (Main Campus) 
Location: Nestled in Ettimadai, this is the largest campus, housing over 12,000 students and 1,500 faculty members.
Academic Range: Offers 120+ programs across UG, PG, and Doctoral levels, including Engineering, Business, and Social Work.
Research: Home to an Internal Quality Assurance Cell (IQAC) and numerous specialized research centers.
Key Contacts: 0422-2685509 / 0422-2685510.

The Coimbatore campus, established in 1994, serves as the administrative headquarters of the university and is a sprawling 400-acre "oasis" nestled 
in the Western Ghats at Ettimadai. As of 2026, it supports over 12,000 students and nearly 1,500 faculty members across 120+ undergraduate,
 postgraduate, and doctoral programs. The campus is renowned for its eco-friendly initiatives, including massive forestation that 
 transformed once-barren land into a lush environment home to exotic birds like peacocks. Infrastructure highlights include a built-up 
 area of 28 lakh square feet, eight hostels accommodating 5,000 students, and an Olympic-standard swimming pool (50m x 25m). 
 Academic life is centered around the Schools of Engineering and Business, alongside Departments of Social Work and Communication. 
 For the 2026 placement season, the campus maintains a strong record with top recruiters like Google, Microsoft, and Intel, reporting 
 a highest package of 60 LPA and an average of 9 LPA.
""",

"""
3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for blending science with spirituality, offering programs in Computing, Engineering, Biotechnology, and Ayurveda.
Key Dates (2026): Even semester classes for 2026 are scheduled to commence on January 19, 2026.
Key Contacts: 7034024264 / 0476-2805280.

Situated in the serene coastal village of Vallikavu, Kerala, the Amritapuri campus spans 80 acres and is a unique blend of
 modern science and spiritual values. It features over one million square feet of built-up area, housing six prestigious schools 
 and more than 20 specialized research centers. The campus is highly residential, with approximately 95 percent of students living in hostels, 
 many of which are located within the ashram premises. Academic programs cover diverse fields including Biotechnology, Ayurveda, Engineering,
   and Social Sciences. Amritapuri is a research powerhouse, hosting the Humanitarian Technology (HuT) Labs, which recently launched the IOT-based 
   "PRABHA" robot and is hosting the 2026 IEEE International Conference on Robotics and Mechatronics. Facilities for students include high-speed 
   computer labs with over 600 systems, extensive sports complexes for cricket and football, and strictly vegetarian dining that promotes a holistic
     lifestyle.

"""



]

In [7]:
import os

data_dir = "data1"
os.makedirs(data_dir, exist_ok=True)

for i, doc in enumerate(sample_docs):
    with open(f"{data_dir}/doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(doc)

print("Documents saved in:", data_dir)


Documents saved in: data1


In [8]:
#STEP1:  -----> DOCUMENT LOADING
from langchain_community.document_loaders import DirectoryLoader,TextLoader

# Load documents from directory
loader = DirectoryLoader(
    "data1", 
    glob="*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview:")
print(documents[0].page_content[:200] + "...")


Loaded 3 documents

First document preview:

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tec...


In [9]:
documents

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="\n1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\n For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B

In [10]:
#step2----->DOCUMENT SPLITTING
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,
    separators=[" "]  # Hierarchy of separators
)
chunks=text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 11 chunks from 3 documents

Chunk example:
Content: 1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academi...
Metadata: {'source': 'data1\\doc_0.txt'}


In [11]:
chunks

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content='1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre'),
 Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="/ 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\

In [16]:

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name="amrita_clg"
)

vectorstore.persist()

print("Vectors stored:", vectorstore._collection.count())


Vectors stored: 11


C:\Users\DELL\AppData\Local\Temp\ipykernel_29208\1931119460.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
#just to load--without duplicating the vectors

persist_directory = "./chroma_db"

# Load existing vectorstore
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=OpenAIEmbeddings(),
    collection_name="amrita_clg"
)

print(f"Vector store has {vectorstore._collection.count()} vectors")


Vector store has 11 vectors


C:\Users\DELL\AppData\Local\Temp\ipykernel_29208\3512872745.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [18]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model_name="gpt-5-nano"
)


In [19]:
## Convert vector store to retriever
retriever=vectorstore.as_retriever(
    search_kwargs={"k":3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B417EA79D0>, search_kwargs={'k': 3})

In [20]:
##CREATING RAG CHAIN USING LCEL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [21]:
# Create a custom prompt
from langchain_core.prompts import ChatPromptTemplate
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question 
about the college and place. Donot hallucinate and do not guess anything.
be crystal clear to the asked question.
If you don't know the answer based on the context, say you don't know.
give the specific details and support.

Context:
{context}

Question: {query}

Answer:""")
custom_prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])

In [22]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B417EA79D0>, search_kwargs={'k': 3})

In [23]:
#output doc for the promts...(combining all the chunks from the retriever for the prompt.)
def combi(docs):
    return "\n".join(doc.page_content for doc in docs)


In [24]:
rag_chain_lcel=(
    {
        "context":retriever|combi,
        "query":RunnablePassthrough()
    }
    |custom_prompt
    |llm
    |StrOutputParser()
)
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B417EA79D0>, search_kwargs={'k': 3})
           | RunnableLambda(combi),
  query: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': Fal

In [25]:
response=rag_chain_lcel.invoke("what is the location on amritapuri campus in 1000 words")
response

'The Amritapuri campus is located in Vallikavu, a serene coastal village in the state of Kerala, India. The location is described in the materials as “Located in Vallikavu, Kerala, spanning 80 acres along the coastline,” which places the campus directly on the coastal belt of the region. In other words, the campus sits beside the sea within the rural and tranquil setting of Vallikavu, a village noted for its coastal character and peaceful surroundings.\n\nKey aspects of the location that define the campus’s sense of place are as follows:\n\n- Geographic setting: The campus is situated in Vallikavu, a village on the Kerala coast. The wording emphasizes that it lies along the coastline, indicating direct proximity to the sea and exposure to coastal scenery. This coastal placement is a defining feature of the campus location, contributing to the campus atmosphere and environment.\n\n- Physical extent: The campus covers 80 acres of land. This sizeable area indicates a substantial, spread-o

In [26]:
#to test query_rag_lcel()

In [27]:
from IPython.display import display, Markdown

display(Markdown(response))


The Amritapuri campus is located in Vallikavu, a serene coastal village in the state of Kerala, India. The location is described in the materials as “Located in Vallikavu, Kerala, spanning 80 acres along the coastline,” which places the campus directly on the coastal belt of the region. In other words, the campus sits beside the sea within the rural and tranquil setting of Vallikavu, a village noted for its coastal character and peaceful surroundings.

Key aspects of the location that define the campus’s sense of place are as follows:

- Geographic setting: The campus is situated in Vallikavu, a village on the Kerala coast. The wording emphasizes that it lies along the coastline, indicating direct proximity to the sea and exposure to coastal scenery. This coastal placement is a defining feature of the campus location, contributing to the campus atmosphere and environment.

- Physical extent: The campus covers 80 acres of land. This sizeable area indicates a substantial, spread-out campus rather than a compact, urban campus. The large footprint supports a diverse campus ecosystem, including academic facilities, residential space, and research infrastructure, all embedded within a single coastal setting.

- Built environment: The Amritapuri campus has over one million square feet of built-up area. This figure signals a dense and extensive built environment within the 80-acre site, reflecting a substantial concentration of classrooms, laboratories, administrative buildings, housing, and other facilities, all arranged to fit within the coastal landscape of Vallikavu.

- Campus composition and use: The site houses six prestigious schools and more than 20 specialized research centers. This mix of schools and research facilities indicates that the location is a multi-disciplinary academic hub within Vallikavu, designed to accommodate a broad spectrum of disciplines in close proximity to one another on the same campus grounds.

- Residential character: The campus is described as highly residential, with approximately 95 percent of students living in hostels, many of which are located within the ashram premises. This aspect of the location highlights a self-contained community where living spaces are integrated with the academic and spiritual environment. The residential nature reinforces the sense that the campus is a self-sustaining micro-community situated in Vallikavu’s coastal landscape.

- Spiritual-educational blend: The location is characterized by a blend of modern science and spiritual values. While this speaks to the campus’s ethos rather than a strictly geographic attribute, it is a defining feature of how the campus sits in its physical space and how students experience the location daily. The coastal setting in Vallikavu is presented as part of the overall environment that supports this unique fusion of disciplines and perspectives.

- Surrounding facilities and capacity: With over 20 specialized research centers and facilities such as HuT Labs (Humanitarian Technology Labs) and other advanced laboratories, the campus location supports a research-intensive environment. The coastal campus arrangement allows these facilities to occupy distinct zones within the 80-acre site, contributing to a sense of breadth and diversity in the location itself.

- Community and accessibility context: The location within a serene coastal village suggests a tranquil, less urbanized surroundings that contrast with more crowded urban campuses. Being in Vallikavu places the Amritapuri campus in a locale that is described as serene and coastal, which can influence factors such as campus life, pace of daily routines, and the way students perceive and experience the location during their studies.

In summary, the location of the Amritapuri campus is defined by its setting in Vallikavu, Kerala, along the coastline. It occupies 80 acres of coastal land and contains over one million square feet of built-up space, organized to accommodate six schools and more than 20 research centers. The campus is distinctly residential, with the majority of students living in on-campus hostels, many of which lie within the ashram premises. This combination of a large, coastal, self-contained site with a multi-disciplinary academic and research ecosystem creates a unique location that blends modern science with spiritual values, all situated in the serene coastal environment of Vallikavu. If you are seeking the precise geographic placement, the official description provides Vallikavu, Kerala as the location, explicitly tied to the coastal landscape and the extensive campus footprint described above.

In [28]:
def query_rag(question):
    print(f"Question:{question}")
    print("-"*50)

    #1)pass the string directly (when using RunnablePassThrough)
    ans=rag_chain_lcel.invoke(question)
    #print(f"answer:{ans}")
    display(Markdown(ans))

    #2)printing the sources for the confirmation
    docs=retriever.invoke(question)
    for i,doc in enumerate(docs):
      print(f"\n source{i+1} \n")
      print(doc.page_content[:200])
    
    

In [29]:
res=query_rag(
  "Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words"
)


Question:Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words
--------------------------------------------------


Amrita Vishwa Vidyapeetham Bengaluru Campus, also known as the Bengaluru Campus of the Amrita Vishwa Vidyapeetham system, is located off Sarjapur Road in the Kasavanahalli/Junnasandra area of Bengaluru. The campus sits on a spacious 50-acre site that is described as a dynamic tech hub designed to cater to the needs of India’s Silicon Valley. The location places it in the rapidly developing tech corridor near Bengaluru’s eastern periphery, offering a practical setting for engineering and computing education.

Location and campus footprint
- Area: 50 acres
- Locality: Off Sarjapur Road in Kasavanahalli/Junnasandra, Bengaluru
- Character: A technology-focused campus built to support advanced engineering and computing programs, reflecting the campus’s emphasis on modern, industry-relevant education.

Academic focus and programs
- School of Engineering & Computing: The Bengaluru Campus houses engineering and computing disciplines with a clear emphasis on high-demand technical fields.
- B.Tech programs: Offered in five disciplines. The precise disciplines are not listed in the provided context, but the programs are described as varied within engineering.
- M.Tech programs: Offered in six disciplines.
- Core focus areas: The campus places special emphasis on Computer Science, Artificial Intelligence, and Cybersecurity, aligning with current industry needs and research trends.
- Departments: Key departments include Engineering and Computing, supporting a range of tech-centric curricula and research initiatives.

Connectivity and access
- Distance from central transit hubs: Approximately 17 kilometers from Majestic Bus Terminus/City Railway Station, making the campus reachable from central Bengaluru by road.
- Public transport routes: Accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market), providing direct connectivity to students and staff traveling from various parts of the city.
- Practical note: The campus’s placement within Bengaluru’s tech belt and its road access via major BMTC routes facilitate easy daily commuting for campus residents and visitors.

Key contacts
- Primary contact numbers: 7022588703 and 6366576503
- These numbers are listed as channels for inquiries related to admissions, programs, placements, and campus information.

Placements and outcomes (as reported for 2026)
- Placement highlights: For 2026, the Bengaluru Campus reported exceptional placement statistics.
- Highest packages: Reported to reach up to 80 LPA for B.Tech students, indicating strong industry demand and competitive offers for graduates from this campus.
- Implication: The high end of the placement statistics suggests robust relationships with recruiters and a focus on high-value technical roles.

Overall characterization
- The Bengaluru Campus of Amrita Vishwa Vidyapeetham is presented as a 50-acre, technology-forward campus located on Sarjapur Road in Kasavanahalli/Junnasandra. Its emphasis on engineering and computing, combined with a strategic location in Bengaluru’s Silicon Valley region, positions it as a hub for technical education and career-oriented outcomes. The campus’s program mix—B.Tech in multiple disciplines and M.Tech in several areas—along with a stated focus on Computer Science, AI, and Cybersecurity, aligns with contemporary industry needs. Availability of direct BMTC connections to major city hubs makes access convenient for students commuting from across Bengaluru. For anyone seeking detailed program lists, admission specifics, or current year intake, contacting the provided numbers would be the recommended first step.


 source1 

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech

 source2 

/ 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand te

 source3 

3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for b


#adding documents (updating)

In [30]:
new_info="""

Amrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students 
across its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students 
whose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in
 four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or double-occupancy
   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. 
The hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and 
is supervised by a team of wardens and faculty wardens who monitor student well-being and activities. All hostels are equipped
 with amenities such as Wi-Fi, solar water heaters for hot water, and a dedicated study hall or reading room. A major highlight
   (or point of note) is that only pure vegetarian food is served in the mess halls, 
   with non-vegetarian food strictly prohibited within the entire campus premises.
     While some students find the food quality average due to a repeating menu, night cafeterias are available for additional
       options. Strict rules are in place regarding timings, outings (usually limited passes per month), and the prohibition
         of intoxicants, tobacco, and specific electrical appliances like kettles or iron boxes within rooms.
           The annual hostel fees vary, ranging approximately from INR 77,000 to INR 121,000 including mess charges, 
           depending on the campus and room type. Students can find more detailed information and apply through the 
           official Amrita Vishwa Vidyapeetham website.
"""

In [31]:
new_docs=Document(
    page_content=new_info,
    metadata={"source":"manual updation", "topic":"amrita hostels"}

)
new_docs

Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='\n\nAmrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students \nacross its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students \nwhose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in\n four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or double-occupancy\n   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. \nThe hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and \nis supervised by a team of wardens and faculty wardens who monitor student well-being and activities. All hostels are equipped\n with amenities such as Wi-Fi,

In [32]:
new_chunks=text_splitter.split_documents([new_docs])
new_chunks

[Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='Amrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female students \nacross its various campuses, including Bengaluru and Coimbatore. Staying in the hostel is generally mandatory for all students \nwhose parents do not reside within a 30 km radius of the campus. The rooms are typically shared, with undergraduates often in\n four-sharing rooms during their initial years and postgraduate/final-year students eligible for single or'),
 Document(metadata={'source': 'manual updation', 'topic': 'amrita hostels'}, page_content='students eligible for single or double-occupancy\n   rooms based on availability. Each room is furnished with a cot, table, chair, and cupboard for each student. \nThe hostel environment emphasizes a disciplined and holistic lifestyle, incorporating activities like yoga and sports, and \nis supervised by a team of wardens 

In [33]:
vectorstore

In [34]:
vectorstore.add_documents(new_chunks)

['a9f79d9f-a6fe-4eb1-b4ae-6988c8fb01c6',
 'cb0d21f4-d6b5-4716-a586-840e84125330',
 '6754cda0-c62f-4342-a36e-cda26b5bb2e3',
 '87de241a-b6d1-4e5a-9ff8-09b249dbffca']

In [35]:
print(f"vectorstore has {vectorstore._collection.count()} vectors")

vectorstore has 15 vectors


In [36]:
new_query="give me information about amrita hostels"
answer=rag_chain_lcel.invoke(new_query)
display(Markdown(answer))

Here are the key details about Amrita hostels as described in the provided context:

- Availability and coverage
  - Amrita Vishwa Vidyapeetham offers hostel facilities across its campuses, including Bengaluru and Coimbatore.
  - The Amritapuri (Vallikavu, Kerala) campus is highly residential with a large hostel presence.

- Mandatory housing
  - Staying in the hostel is generally mandatory for students whose parents do not reside within 30 km of the campus.

- Room types and occupancy
  - Undergraduates typically stay in four-sharing rooms during their initial years.
  - Postgraduate and/or final-year students may be eligible for single rooms or limited monthly passes.

- Rules and restrictions
  - Prohibition of intoxicants and tobacco.
  - Restrictions on certain electrical appliances inside rooms (e.g., kettles or iron boxes).

- Fees
  - The annual hostel fees range roughly from INR 77,000 to INR 121,000, including mess charges. The exact amount depends on the campus and room type.

- How to learn more or apply
  - Detailed information and the application process are available on the official Amrita Vishwa Vidyapeetham website.

- Amritapuri campus specifics (Vallikavu, Kerala)
  - The campus spans about 80 acres and is described as a unique blend of modern science and spiritual values.
  - It is highly residential, with approximately 95% of students living in hostels, many of which are located within the ashram premises.

#building rag- Conversational Memory

In [47]:
# Updated 2026 Imports
from langchain_classic.chains import create_history_aware_retriever
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain


from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage


In [40]:
## create a prompt that includes the chat history
context_system_prompt = """Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question 
which can be understood without the chat history. Do NOT answer the question, 
just reformulate it if needed and otherwise return it as is."""

context_prompt = ChatPromptTemplate.from_messages([
    ("system", context_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [41]:
## create history aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, context_prompt
)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B417EA79D0>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(ta

In [73]:
print("""
1.LLM writes a stand alone question from chat history to retrieve proper data from the vectorstore
2.LLM generates answer in the second prompt (uses chat history)
      """)


1.LLM writes a stand alone question from chat history to retrieve proper data from the vectorstore
2.LLM generates answer in the second prompt (uses chat history)
      


In [74]:
print(
"""llm creates stand alone question to send it to the chromadb
create_history_aware_retriever --->retrieves vectors
create_stuff_documents_chain --->connects the llm, prompt
rag_chain-->connects the previous one to the history_aware_retriever""")

llm creates stand alone question to send it to the chromadb
create_history_aware_retriever --->retrieves vectors
create_stuff_documents_chain --->connects the llm, prompt
rag_chain-->connects the previous one to the history_aware_retriever


In [66]:
# Create a new document chain with history
userquery_system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

user_prompt = ChatPromptTemplate.from_messages([
    ("system", userquery_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, user_prompt)

# Create conversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever, 
    question_answer_chain
)
print("Conversational RAG chain created!")

Conversational RAG chain created!


In [67]:
chat_history=[]

result1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "how do the hostels in amrita look like?"
})
print(f"Q: how do the hostels in amrita look like?")
print(f"A: {result1['answer']}")
display(Markdown(result1['answer']))

Q: how do the hostels in amrita look like?
A: Amrita hostels are gender-segregated dormitories across its campuses (including Bengaluru and Coimbatore). Rooms are furnished with a cot, table, chair, and cupboard; undergrads typically stay in four-sharing rooms, while postgraduates may have single or double occupancy based on availability. The facilities include Wi‑Fi, solar water heaters, wardens, and a disciplined, holistic environment with activities like yoga and sports; annual hostel fees are about INR 77,000–121,000 including mess charges.


Amrita hostels are gender-segregated dormitories across its campuses (including Bengaluru and Coimbatore). Rooms are furnished with a cot, table, chair, and cupboard; undergrads typically stay in four-sharing rooms, while postgraduates may have single or double occupancy based on availability. The facilities include Wi‑Fi, solar water heaters, wardens, and a disciplined, holistic environment with activities like yoga and sports; annual hostel fees are about INR 77,000–121,000 including mess charges.

In [68]:
chat_history.extend([
    HumanMessage(content="how do the hostels in amrita look like?"),
    AIMessage(content=result1['answer'])
])

In [70]:
chat_history

[HumanMessage(content='how do the hostels in amrita look like?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Amrita hostels are gender-segregated dormitories across its campuses (including Bengaluru and Coimbatore). Rooms are furnished with a cot, table, chair, and cupboard; undergrads typically stay in four-sharing rooms, while postgraduates may have single or double occupancy based on availability. The facilities include Wi‑Fi, solar water heaters, wardens, and a disciplined, holistic environment with activities like yoga and sports; annual hostel fees are about INR 77,000–121,000 including mess charges.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [ ]:
## Follow up question
# Follow-up question
result2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "what am i talking abt?" 
})
result2

{'chat_history': [HumanMessage(content='how do the hostels in amrita look like?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Amrita hostels are gender-segregated dormitories across its campuses (including Bengaluru and Coimbatore). Rooms are furnished with a cot, table, chair, and cupboard; undergrads typically stay in four-sharing rooms, while postgraduates may have single or double occupancy based on availability. The facilities include Wi‑Fi, solar water heaters, wardens, and a disciplined, holistic environment with activities like yoga and sports; annual hostel fees are about INR 77,000–121,000 including mess charges.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])],
 'input': 'what am i talking abt?',
 'context': [Document(metadata={'topic': 'amrita hostels', 'source': 'manual updation'}, page_content='Amrita Vishwa Vidyapeetham provides comprehensive hostel facilities, with separate accommodation for male and female stude

In [72]:
from IPython.display import Markdown,display
result2['answer']
display(Markdown(result2['answer']))

You're talking about Amrita Vishwa Vidyapeetham's hostel facilities across its campuses, including Bengaluru, Coimbatore, and Amritapuri. It covers room types, rules, and approximate annual fees.